In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

import nest_asyncio


sys.path.insert(0, os.path.abspath('..'))
nest_asyncio.apply()

In [1]:
100 * 1024 * 1024

104857600

In [2]:
import logging


logging.basicConfig(
    level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s'
)
logging.getLogger('pylatexenc.latexwalker').setLevel(logging.ERROR)
logging.getLogger('httpx').setLevel(logging.WARNING)
logging.getLogger('openai').setLevel(logging.WARNING)

In [3]:
from math_rag.infrastructure.containers import InfrastructureContainer


RESET = False

infrastructure_container = InfrastructureContainer()
infrastructure_container.init_resources()

# seeders
math_article_seeder = infrastructure_container.math_article_seeder()
math_expression_seeder = infrastructure_container.math_expression_seeder()
math_expression_classification_seeder = (
    infrastructure_container.math_expression_classification_seeder()
)
em_failed_request_seeder = infrastructure_container.em_failed_request_seeder()
llm_failed_request_seeder = infrastructure_container.llm_failed_request_seeder()

math_article_seeder.seed(reset=RESET)
await math_expression_seeder.seed(reset=RESET)
await math_expression_classification_seeder.seed(reset=RESET)
await em_failed_request_seeder.seed(reset=RESET)
await llm_failed_request_seeder.seed(reset=RESET)

# repositories
math_article_repository = infrastructure_container.math_article_repository()
math_expression_repository = infrastructure_container.math_expression_repository()
math_expression_classification_repository = (
    infrastructure_container.math_expression_classification_repository()
)
google_file_repository = infrastructure_container.google_file_repository()

# other
katex_corrector_assistant = infrastructure_container.katex_corrector_assistant()

2025-05-20 15:08:18,506 - INFO - PyTorch version 2.6.0 available.
2025-05-20 15:08:19,046 - INFO - file_cache is only supported with oauth2client<4.0.0


In [ ]:
# await math_expression_seeder.seed(reset=True)
# await llm_failed_request_seeder.seed(reset=True)

### Load

In [ ]:
from math_rag.application.enums.arxiv import MathCategory


math_article_loader_service = infrastructure_container.math_article_loader_service()
# await math_article_loader_service.load(MathCategory, 200)

### Parse

In [ ]:
math_expression_loader_service = (
    infrastructure_container.math_expression_loader_service()
)

# await math_expression_loader_service.load()

### Label

### Display

In [ ]:
from IPython.display import Math, display


for i, latex_math_node in enumerate(math_nodes[:100]):
    latex = latex_math_node.latex_verbatim()
    math_display_object = Math(latex)

    display(math_display_object)

### Correct

In [6]:
incorrect_katex = r'd\omega = \theta \w \omega'
error = r'KaTeX parse error: Undefined control sequence: \w at position 18: …omega = \theta \̲w̲ ̲\omega'

In [7]:
from math_rag.application.models.assistants import KatexCorrectorAssistantInput


input = KatexCorrectorAssistantInput(katex=incorrect_katex, error=error)

In [ ]:
output = await katex_corrector_assistant.assist(input)
corrected_katex = output.katex
print(corrected_katex)
display(Math(corrected_katex))

In [ ]:
import copy


inputs = [copy.deepcopy(input) for _ in range(5000)]
outputs = await katex_corrector_assistant.concurrent_assist(inputs)

### Dataset

#### Upload

In [4]:
from math_rag.application.models.datasets import (
    MathExpressionDataset,
    MathExpressionSample,
)
from math_rag.core.enums import MathExpressionLabelEnum


samples = [
    MathExpressionSample(
        latex=f'x + {i} = 5',
        label=MathExpressionLabelEnum.EQUALITY,
    )
    for i in range(10)
]
dataset = MathExpressionDataset(samples)

In [5]:
from math_rag.application.assistants.prompts import MATH_EXPRESSION_LABELER_PROMPT
from math_rag.application.models.datasets import (
    DatasetMetadataFile,
    DatasetSplitSettings,
)


settings = DatasetSplitSettings(
    train_ratio=0.8, validate_ratio=0.1, test_ratio=0.1, seed=42
)

json_str = MATH_EXPRESSION_LABELER_PROMPT.model_dump_json(indent=4)
content = json_str.encode('utf-8')
metadata_file = DatasetMetadataFile(name='prompt.json', content=content)

In [6]:
dataset_publisher_service = infrastructure_container.dataset_publisher_service()
dataset_publisher_service.publish(
    dataset, MathExpressionSample, settings, metadata_file
)

2025-05-14 21:12:31,414 - INFO - Dataset KebabSeller/mathexpressiondataset already exists


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

#### Download

In [7]:
from datasets import load_dataset
from datasets.download import DownloadConfig
from decouple import config


HF_USERNAME = config('HF_USERNAME', default=None)
HF_TOKEN = config('HF_TOKEN', default=None)

download_config = DownloadConfig(
    max_retries=3,
    disable_tqdm=True,
)

dataset_dict = load_dataset(
    path=f'{HF_USERNAME}/mathexpressiondataset',
    split=None,
    download_config=download_config,
    token=HF_TOKEN,
    trust_remote_code=True,
)

README.md:   0%|          | 0.00/641 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

validate-00000-of-00001.parquet:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8 [00:00<?, ? examples/s]

Generating validate split:   0%|          | 0/1 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1 [00:00<?, ? examples/s]

In [8]:
from typing import cast

from datasets import ClassLabel


class_label = cast(ClassLabel, dataset_dict['train'].features['label'])
class_label.names

['equality', 'inequality', 'constant', 'variable', 'other']